In [1]:
import os
import sys

#this part is used for pyspark submit
os.environ['PYSPARK_SUBMIT_ARGS']='\
--verbose \
--master=yarn \
--deploy-mode=client \
pyspark-shell'

os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['YARN_CONF_DIR']='/etc/alternatives/hadoop-conf/'

#this line is used for spark1.6
#os.environ['SPARK_HOME']='/opt/cloudera/parcels/CDH/lib/spark'

#this line is used for spark2.2
os.environ['SPARK_HOME']='/opt/cloudera/parcels/SPARK2-2.2.0.cloudera2-1.cdh5.12.0.p0.232957/lib/spark2'

# this line is used for python2.7
#os.environ['PYSPARK_PYTHON']='/usr/bin/python'

#this line is used for python3.5
os.environ['PYSPARK_PYTHON']='/usr/bin/python3'

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))  
#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0.cloudera2
      /_/

Using Python version 3.5.2 (default, Nov 23 2017 16:37:01)
SparkSession available as 'spark'.


In [2]:
import pyspark
sc_ = SparkSession.builder.appName('fuck').getOrCreate()

df_WAT = sc_.read.format('csv').option('header', 'true').load('/user/team6/homework_1/WAT.csv')
df_WAT.head(1)
df_yield = sc_.read.format('csv').option('header', 'true').load('/user/team6/homework_1/Yield.csv')
df_yield.head(1)


[Row(wafer_id='LT0001.01', yield='76.9795724366398')]

In [3]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler

merge_df = df_yield.join(df_WAT, 'wafer_id', 'inner').drop('wafer_id').drop(df_WAT._c0)
merge_df.printSchema()

#columns_name = df.columns
columns_name = ["yield","WAT1","WAT2","WAT3","WAT4","WAT5"]
typechange = [col(column).cast("Double").alias(column) for column in columns_name]
typechange_df = merge_df.select(typechange)
typechange_df.printSchema()
assembler = VectorAssembler(inputCols=columns_name,outputCol="features")
output = assembler.transform(typechange_df).select("features")
output.show()


root
 |-- yield: string (nullable = true)
 |-- WAT1: string (nullable = true)
 |-- WAT2: string (nullable = true)
 |-- WAT3: string (nullable = true)
 |-- WAT4: string (nullable = true)
 |-- WAT5: string (nullable = true)
 |-- WAT6: string (nullable = true)
 |-- WAT7: string (nullable = true)
 |-- WAT8: string (nullable = true)
 |-- WAT9: string (nullable = true)
 |-- WAT10: string (nullable = true)
 |-- WAT11: string (nullable = true)
 |-- WAT12: string (nullable = true)
 |-- WAT13: string (nullable = true)
 |-- WAT14: string (nullable = true)
 |-- WAT15: string (nullable = true)
 |-- WAT16: string (nullable = true)
 |-- WAT17: string (nullable = true)
 |-- WAT18: string (nullable = true)
 |-- WAT19: string (nullable = true)
 |-- WAT20: string (nullable = true)
 |-- WAT21: string (nullable = true)
 |-- WAT22: string (nullable = true)
 |-- WAT23: string (nullable = true)
 |-- WAT24: string (nullable = true)
 |-- WAT25: string (nullable = true)
 |-- WAT26: string (nullable = true)
 |-- 

root
 |-- yield: double (nullable = true)
 |-- WAT1: double (nullable = true)
 |-- WAT2: double (nullable = true)
 |-- WAT3: double (nullable = true)
 |-- WAT4: double (nullable = true)
 |-- WAT5: double (nullable = true)

+--------------------+
|            features|
+--------------------+
|[76.9795724366398...|
|[75.2029172318689...|
|[77.0216726433812...|
|[75.2023758930297...|
|[79.2773721175721...|
|[77.2425479387875...|
|[76.4054086079774...|
|[78.217875571929,...|
|[75.5875504902877...|
|[75.2420699261284...|
|[75.3914766769037...|
|[76.1083246485199...|
|[72.2381387653298...|
|[75.9324870525181...|
|[80.6229895244045...|
|[77.944680338383,...|
|[77.9292160824658...|
|[74.9117349980161...|
|[77.3424464969162...|
|[77.001637342853,...|
+--------------------+
only showing top 20 rows



In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

r1 = Correlation.corr(output, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))




Pearson correlation matrix:
DenseMatrix([[1.        , 0.3641    , 0.2456    , 0.4573    , 0.4453    ,
              0.4706    ],
             [0.3641    , 1.        , 0.09509106, 0.17808983, 0.16892587,
              0.1691694 ],
             [0.2456    , 0.09509106, 1.        , 0.07408941, 0.11578342,
              0.11078233],
             [0.4573    , 0.17808983, 0.07408941, 1.        , 0.21984015,
              0.2368346 ],
             [0.4453    , 0.16892587, 0.11578342, 0.21984015, 1.        ,
              0.21840688],
             [0.4706    , 0.1691694 , 0.11078233, 0.2368346 , 0.21840688,
              1.        ]])
